In [41]:
import zipfile
from PIL import Image
import numpy as np
import pytesseract
import math
import os
from mtcnn import MTCNN
import tensorflow as tf
import cv2 as cv
import time



In [42]:
from PIL.ImageDraw import ImageDraw

# Loading the face detection classifier
face_cascade = cv.CascadeClassifier('/Users/yatinpatel/Desktop/Python3-Programming-Course/haarcascade_frontalface_default.xml')

# Define the path to the pre-trained MTCNN model
mtcnn_model_path = '/Users/yatinpatel/Desktop/Python3-Programming-Course/mtcnn_weights.npy'

# Create the MTCNN face detector
face_detector = MTCNN(weights_file=mtcnn_model_path)

# Define the output folder for Haar Cascade Classifier
output_folder_haar = '/Users/yatinpatel/Desktop/Python3-Programming-Course/Haar_output_boundingbox'

# Create the output folder for Haar Cascade Classifier if it doesn't exist
if not os.path.exists(output_folder_haar):
    os.makedirs(output_folder_haar)

# Define the output folder for MTCNN
output_folder_mtcnn = '/Users/yatinpatel/Desktop/Python3-Programming-Course/MTCNN_output_boundingbox'

# Create the output folder for MTCNN if it doesn't exist
if not os.path.exists(output_folder_mtcnn):
    os.makedirs(output_folder_mtcnn)

# Function to calculate processing time for Haar Cascade Classifier
def get_processing_time_haar(img, scale_factor):
    start_time = time.time()
    gray = np.array(img.convert("L"))
    face_cascade.detectMultiScale(gray, scale_factor)
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time

# Function to calculate processing time for MTCNN
def get_processing_time_mtcnn(img):
    start_time = time.time()
    face_detector.detect_faces(np.array(img))
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time
# Define a function to extract faces using MTCNN and create a contact sheet
def extract_faces_mtcnn(img):
    # Convert PIL image to numpy array
    img_np = np.array(img)

    # Perform face detection
    faces = face_detector.detect_faces(img_np)

    # If no faces are detected
    if len(faces) == 0:
        return None

    # Extract faces into the list
    faces_imgs = []

    for face in faces:
        x, y, w, h = face['box']
        face_img = img.crop((x, y, x + w, y + h))
        faces_imgs.append((face_img, face['confidence']))

    # Compute nrows and ncols
    ncols = 5
    nrows = math.ceil(len(faces_imgs) / ncols)

    # Create a contact sheet
    contact_sheet = Image.new(img.mode, (550, 110 * nrows))
    x, y = (0, 0)

    for face, confidence in faces_imgs:
        face.thumbnail((110, 110))
        contact_sheet.paste(face, (x, y))
        draw = ImageDraw.Draw(contact_sheet)
        draw.text((x + 5, y + 5), f'Confidence: {confidence:.2f}', (255, 255, 255))

        if x + 110 == contact_sheet.width:
            x = 0
            y += 110
        else:
            x += 110

    return contact_sheet

# Define a function to extract faces using Haar Cascade Classifier and create a contact sheet
def extract_faces_haar(img, scale_factor):
    # Extract the rectangles of the faces
    gray = np.array(img.convert("L"))
    faces = face_cascade.detectMultiScale(gray, scale_factor)

    # If no faces are detected
    if len(faces) == 0:
        return None

    # Extract faces into the list
    faces_imgs = []

    for x, y, w, h in faces:
        faces_imgs.append(img.crop((x, y, x + w, y + h)))

    # Compute nrows and ncols
    ncols = 5
    nrows = math.ceil(len(faces_imgs) / ncols)

    # Create a contact sheet
    contact_sheet = Image.new(img.mode, (550, 110 * nrows))
    x, y = (0, 0)

    for face in faces_imgs:
        face.thumbnail((110, 110))
        contact_sheet.paste(face, (x, y))

        if x + 110 == contact_sheet.width:
            x = 0
            y += 110
        else:
            x += 110

    return contact_sheet


In [43]:
# Define function to detect faces using Haar Cascade Classifier
def detect_faces_haar(img, scale_factor):
    gray = np.array(img.convert("L"))
    faces = face_cascade.detectMultiScale(gray, scale_factor)
    if len(faces) == 0:
        return None
    else:
        for (x, y, w, h) in faces:
            cv.rectangle(np.array(img), (x, y), (x + w, y + h), (0, 255, 0), 2)
        return img

# Define function to detect faces using MTCNN
def detect_faces_mtcnn(img):
    faces = face_detector.detect_faces(np.array(img))
    if len(faces) == 0:
        return None
    else:
        for face in faces:
            x, y, w, h = face['box']
            cv.rectangle(np.array(img), (x, y), (x + w, y + h), (0, 255, 0), 2)
        return img

# Define function to get processing time for Haar Cascade Classifier
def get_processing_time_haar(img, scale_factor):
    start_time = time.time()
    detect_faces_haar(img, scale_factor)
    end_time = time.time()
    return end_time - start_time

# Define function to get processing time for MTCNN
def get_processing_time_mtcnn(img):
    start_time = time.time()
    detect_faces_mtcnn(img)
    end_time = time.time()
    return end_time - start_time

# Define functions to calculate precision and recall
def calculate_precision(true_positives, false_positives):
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0

def calculate_recall(true_positives, total_samples):
    return true_positives / total_samples if total_samples > 0 else 0.0


In [44]:
# Define image folder path
image_folder = '/Users/yatinpatel/Desktop/Python3-Programming-Course/extracted_images'

# Get the list of image files
image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]


# Calculate total number of images
total_images = len(image_files)

# Lists to store processing time for each image
total_time_haar = []
total_time_mtcnn = []

# Calculate total true positives, false positives, and false negatives for each model
total_true_positives_haar = 0
total_false_positives_haar = 0
total_false_negatives_haar = 0
total_true_positives_mtcnn = 0
total_false_positives_mtcnn = 0
total_false_negatives_mtcnn = 0

# Function to draw bounding boxes for Haar Cascade Classifier
def draw_bounding_boxes_haar(img, scale_factor):
    gray = np.array(img.convert("L"))
    faces = face_cascade.detectMultiScale(gray, scale_factor)

    for x, y, w, h in faces:
        cv.rectangle(np.array(img), (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img

# Function to draw bounding boxes for MTCNN
def draw_bounding_boxes_mtcnn(img):
    faces = face_detector.detect_faces(np.array(img))

    for face in faces:
        x, y, w, h = face['box']
        cv.rectangle(np.array(img), (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img

# Loop through all images
for image_file in image_files:
    # Load image from the image file
    img = Image.open(os.path.join(image_folder, image_file))

    # Run face detection for Haar Cascade Classifier
    processed_img_haar = detect_faces_haar(img, scale_factor=1.4)
    if processed_img_haar is not None:
        processed_img_haar.save(os.path.join(output_folder_haar, image_file))
        total_time_haar.append(get_processing_time_haar(img, scale_factor=1.4))
        total_true_positives_haar += 1
    else:
        total_false_negatives_haar += 1

    # Run face detection for MTCNN
    processed_img_mtcnn = detect_faces_mtcnn(img)
    if processed_img_mtcnn is not None:
        processed_img_mtcnn.save(os.path.join(output_folder_mtcnn, image_file))
        total_time_mtcnn.append(get_processing_time_mtcnn(img))
        total_true_positives_mtcnn += 1
    else:
        total_false_negatives_mtcnn += 1

# Calculate total detections for each model
total_detections_haar = total_true_positives_haar + total_false_positives_haar
total_detections_mtcnn = total_true_positives_mtcnn + total_false_positives_mtcnn

# Calculate false positives and false negatives for each model
false_positives_haar = total_false_positives_haar
false_negatives_haar = total_false_negatives_haar
false_positives_mtcnn = total_false_positives_mtcnn
false_negatives_mtcnn = total_false_negatives_mtcnn

# Calculate average processing time for Haar Cascade Classifier and MTCNN
avg_time_haar = np.mean(total_time_haar)
avg_time_mtcnn = np.mean(total_time_mtcnn)

# Calculate accuracy, precision, and recall for Haar Cascade Classifier
accuracy_haar = total_true_positives_haar / total_images
precision_haar = calculate_precision(total_true_positives_haar, total_false_positives_haar)
recall_haar = calculate_recall(total_true_positives_haar, total_images)

# Calculate accuracy, precision, and recall for MTCNN
accuracy_mtcnn = total_true_positives_mtcnn / total_images
precision_mtcnn = calculate_precision(total_true_positives_mtcnn, total_false_positives_mtcnn)
recall_mtcnn = calculate_recall(total_true_positives_mtcnn, total_images)

# Print the results for each model
print("Haar Cascade Classifier:")
print(f"Average Processing Time: {avg_time_haar:.4f} seconds")
print(f"Accuracy: {accuracy_haar:.4f}")
print(f"Precision: {precision_haar:.4f}")
print(f"Recall: {recall_haar:.4f}")
print(f"Total Detections: {total_detections_haar}")
print(f"False Positives: {false_positives_haar}")
print(f"False Negatives: {false_negatives_haar}")

print("\nMTCNN:")
print(f"Average Processing Time: {avg_time_mtcnn:.4f} seconds")
print(f"Accuracy: {accuracy_mtcnn:.4f}")
print(f"Precision: {precision_mtcnn:.4f}")
print(f"Recall: {recall_mtcnn:.4f}")
print(f"Total Detections: {total_detections_mtcnn}")
print(f"False Positives: {false_positives_mtcnn}")
print(f"False Negatives: {false_negatives_mtcnn}")


Haar Cascade Classifier:
Average Processing Time: 0.1916 seconds
Accuracy: 0.8095
Precision: 1.0000
Recall: 0.8095
Total Detections: 34
False Positives: 0
False Negatives: 8

MTCNN:
Average Processing Time: 4.7566 seconds
Accuracy: 0.9048
Precision: 1.0000
Recall: 0.9048
Total Detections: 38
False Positives: 0
False Negatives: 4
